In [1]:
import numpy as np

In [44]:
class Sphere:
    def __init__(self, velocity, mass, radius, id):
        self.id       = id 
        self.velocity = velocity
        self.mass     = mass
        self.radius   = radius
    
    def addVelocity(self, impulse):
        self.velocity += impulse/self.mass
        
    def printInfo(self):
        print("ball {} velocity:{}".format(self.id, self.velocity))

class Contact:
    def __init__(self, masterId, slaveId, colMat, velocity, energy):
        self.masterId = masterId
        self.slaveId  = slaveId
        self.colMat   = colMat
        self.velocity = velocity
        self.energy   = energy
        
    def addEnergy(self, deltaE):
        self.energy += deltaE
        if abs(self.energy) < 0.1:
            self.energy = 0.
        
    def relVelocity(self, spheres):
        if self.slaveId != -1:
            self.velocity = spheres[self.masterId].velocity - spheres[self.slaveId].velocity
        else:
            self.velocity = spheres[self.masterId].velocity
            
        if abs(self.velocity) < 0.1:
            self.velocity = 0
            
    def resititution(self, scalar):
        self.energy *= scalar
            
    def printInfo(self):
        print("masterId:{}, slaveId:{}, colMat:{}, velocity:{}, energy:{}".format(self.masterId, self.slaveId, self.colMat, self.velocity, self.energy))

In [45]:
contact1 = Contact(0,-1,0.1,-30,0)
contact2 = Contact(1,0,0.2,-20,0)
contact3 = Contact(2,1,0.2,-10,0)
sphere1  = Sphere(-30,10,10,0)
sphere2  = Sphere(-50,10,10,1)
sphere3  = Sphere(-60,10,10,2)
contactList = [contact1, contact2, contact3]
spheres     = [sphere1, sphere2, sphere3]
#contactList = [contact1]
#spheres     = [sphere1]
#START COLLISION RESOLUTION
minVqueue = []
maxEqueue = []
for contact, i in zip(contactList,np.arange(len(contactList))):
    if contact.velocity < 0.:
        minVqueue.append((i, contact.velocity, contact.energy))
#sort by value
minVqueue = sorted(minVqueue, key = lambda x:x[1], reverse=False)

while len(minVqueue) > 0:
    #COMPRESSION PHASE
    count = 0
    while len(minVqueue) > 0:
        firstItem = minVqueue.pop(0)
        contact = contactList[firstItem[0]]
        deltaV  = -contact.velocity/20
        impulse = deltaV/contact.colMat

        if contact.slaveId != -1:
            spheres[contact.masterId].addVelocity(impulse)
            spheres[contact.slaveId].addVelocity(-impulse)
        else:
            spheres[contact.masterId].addVelocity(impulse)
        contact.relVelocity(spheres)
        deltaE  = 0.5*abs(2*contact.velocity+deltaV)*impulse
        contact.addEnergy(deltaE)
        #contact.printInfo()
        minVqueue = []
        for contact, i in zip(contactList,np.arange(len(contactList))):
            contact.relVelocity(spheres)
            if contact.velocity < 0.:
                minVqueue.append((i, contact.velocity, contact.energy))
        minVqueue = sorted(minVqueue, key = lambda x:x[1], reverse=False)
        count += 1
    print("compression takes {} iteration".format(count))

    # RESTITUTION PHASE
    for contact in contactList:
        contact.resititution(1.)
        
    # RELEASE PHASE
    maxEqueue = []
    for contact, i in zip(contactList,np.arange(len(contactList))):
        contact.relVelocity(spheres)
        if contact.energy > 0:
            maxEqueue.append((i, contact.velocity, contact.energy))
    maxEqueue = sorted(maxEqueue, key = lambda x:x[2], reverse=True)
    count = 0
    while len(maxEqueue) > 0:
        firstItem = maxEqueue.pop(0)
        contact = contactList[firstItem[0]]
        #contact.printInfo()
        maxV    = -contact.velocity + np.sqrt(contact.velocity**2 + 2.*contact.energy*contact.colMat)
        deltaV  = maxV/20
        impulse = deltaV/contact.colMat

        if contact.slaveId != -1:
            spheres[contact.masterId].addVelocity(impulse)
            spheres[contact.slaveId].addVelocity(-impulse)
        else:
            spheres[contact.masterId].addVelocity(impulse)
        contact.relVelocity(spheres)
        deltaE  = 0.5*abs(2*contact.velocity+deltaV)*impulse
        contact.addEnergy(-deltaE)
        maxEqueue = []
        for contact, i in zip(contactList,np.arange(len(contactList))):
            contact.relVelocity(spheres)
            if contact.energy > 0:
                maxEqueue.append((i, contact.velocity, contact.energy))
        maxEqueue = sorted(maxEqueue, key = lambda x:x[2], reverse=True)
        count += 1
    print("release takes {} iteration".format(count))
    

    minVqueue = []
    for contact, i in zip(contactList,np.arange(len(contactList))):
        contact.relVelocity(spheres)
        if contact.velocity < 0.:
            minVqueue.append((i, contact.velocity, contact.energy))
    minVqueue = sorted(minVqueue, key = lambda x:x[1], reverse=False)
    
for contact in contactList:
    contact.printInfo()
for sphere in spheres:
    sphere.printInfo()

compression takes 2313 iteration
release takes 528 iteration
compression takes 108 iteration
release takes 197 iteration
compression takes 28 iteration
release takes 0 iteration
masterId:0, slaveId:-1, colMat:0.1, velocity:21.527351247093005, energy:0.0
masterId:1, slaveId:0, colMat:0.2, velocity:0, energy:0.0
masterId:2, slaveId:1, colMat:0.2, velocity:22.040128139655756, energy:0.0
ball 0 velocity:21.527351247093005
ball 1 velocity:21.430575752225785
ball 2 velocity:43.47070389188154


In [3]:
20**2 + 37**2 + 42**2

3533

In [4]:
10**2 + 30**2 + 60**2

4600